In [3]:
import os
!pip3 install -U pip
!pip3 install -qU \
    transformers==4.31.0 \
    accelerate==0.21.0 \
    einops==0.6.1 \
    langchain==0.0.240 \
    xformers==0.0.20 \
    bitsandbytes==0.41.0

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/50/c2/e06851e8cc28dcad7c155f4753da8833ac06a5c704c109313b8d5a62968a/pip-23.2.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2
    Uninstalling pip-23.2:
      Successfully uninstalled pip-23.2


In [4]:
import torch
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [12]:
model_id = 'meta-llama/Llama-2-13b-chat-hf'

# begin initializing HF items, need auth token for these
hf_auth = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [15]:
# initialize the model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()

ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        

In [21]:
modelDir = os.path.expanduser("~/models")

In [24]:
os.path.join(modelDir, "llama-2-7b-chat.ggmlv3.q8_0.bin")

'/Users/cjwang/models/llama-2-7b-chat.ggmlv3.q8_0.bin'

In [25]:
from llama_cpp import Llama
llm = Llama(model_path=os.path.join(modelDir, "llama-2-7b-chat.ggmlv3.q8_0.bin"),
            n_ctx=8192,
            n_batch=512)


llama.cpp: loading model from /Users/cjwang/models/llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 8192
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 7 (mostly Q8_0)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 6828.73 MB (+ 4096.00 MB per state)
llama_new_context_with_model: kv self size  = 409

In [26]:
output = llm("How are you?",
             max_tokens=-1,
             echo=False,
             temperature=0.2,
             top_p=0.1)


llama_print_timings:        load time = 11043.77 ms
llama_print_timings:      sample time =    83.60 ms /   114 runs   (    0.73 ms per token,  1363.70 tokens per second)
llama_print_timings: prompt eval time = 11043.64 ms /     5 tokens ( 2208.73 ms per token,     0.45 tokens per second)
llama_print_timings:        eval time = 2190951.55 ms /   113 runs   (19388.95 ms per token,     0.05 tokens per second)
llama_print_timings:       total time = 2202540.77 ms


In [27]:
output

{'id': 'cmpl-3f71f75e-3399-4bb0-82bc-a1571ebdc25a',
 'object': 'text_completion',
 'created': 1692666581,
 'model': '/Users/cjwang/models/llama-2-7b-chat.ggmlv3.q8_0.bin',
 'choices': [{'text': " I hope you're doing well.\nI wanted to reach out and see how you're feeling after the recent events in your life. It can be tough when things don't go as planned, but remember that you're strong and capable of getting through anything.\nIf there's anything I can do to help, please don't hesitate to let me know. Whether it's just a listening ear or some practical support, I'm here for you.\nTake care of yourself and stay positive. You got this!",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 5, 'completion_tokens': 113, 'total_tokens': 118}}